In [140]:
import pyPI as pi
import numpy as np
from scipy.linalg import eig
import sympy as sp
from scipy.optimize import minimize

In [141]:
# Vectores de entrada y salida
E_in = pi.polarization_basis_set('L')
S = pi.jones_matrix(np.pi/2, np.pi/8)
E_out = S @ E_in
S

array([[0.85355339-0.14644661j, 0.35355339+0.35355339j],
       [0.35355339+0.35355339j, 0.14644661-0.85355339j]])

In [142]:
# Componentes principale
# Cálculo de autovalores y autovectores
eigenvalues, eigenvectors = eig(S)

# Determinación de alpha a partir del autovector principal
alpha_calculado = np.arctan2(np.real(eigenvectors[1, 0]), np.real(eigenvectors[0, 0]))

# Resultados
#print("Autovector principal (dirección dominante):\n", eigenvectors[:, 0])
print(f"α: {alpha_calculado/np.pi:.4f}π")

α: 0.1250π


In [143]:
# Matriz por minimizacion
optimal_delta_chi, optimal_alpha = pi.birefringence_by_minimization(E_in, E_out)

χ: 0.5000π
α: 0.1250π


In [151]:
M = E_out @ np.linalg.pinv(E_in)  # Uso de la pseudoinversa para mayor robustez
M == S

array([[False, False],
       [False, False]])

In [145]:
# Función de error para minimizar la diferencia entre la salida calculada y la esperada
def error_function(params):
    m_11 = params[0] + 1j * params[1]
    m_12 = params[2] + 1j * params[3]
    m_21 = params[4] + 1j * params[5]
    m_22 = params[6] + 1j * params[7]
    
    M = np.array([[m_11, m_12],
                  [m_21, m_22]], dtype=np.complex128)
    
    result = M @ E_in  # Estado de polarización calculado
    return np.linalg.norm(result - E_out)  # Métrica de error

# Suposición inicial para el proceso de minimización
initial_guess = [0, 0, 0, 0, 0, 0, 0, 0]  # Inicializa todas las componentes en cero

# Proceso de minimización usando el método Nelder-Mead
result = minimize(error_function, initial_guess, method='Nelder-Mead')

# Extraer parámetros optimizados
m_11 = result.x[0] + 1j * result.x[1]
m_12 = result.x[2] + 1j * result.x[3]
m_21 = result.x[4] + 1j * result.x[5]
m_22 = result.x[6] + 1j * result.x[7]

# Matriz M optimizada
M = np.array([[m_11, m_12],
              [m_21, m_22]], dtype=np.complex128)

print("Matriz M optimizada:")
print(M)
print(f"Error mínimo alcanzado: {result.fun}")

Matriz M optimizada:
[[ 0.46102798-0.62217828j  0.82925937-0.03895334j]
 [-0.00711608-0.22346206j  0.72348786-1.21422683j]]
Error mínimo alcanzado: 2.9075595897906606e-05


In [154]:
# Metodos mas robustos?
# Pseudoinversa Regularizada (Recomendada para datos ruidosos o mal condicionados)
import numpy as np
from scipy.linalg import svd

def robust_pinv(E_in, epsilon=1e-10):
    U, S, Vh = svd(E_in)  # Descomposición en valores singulares
    S_inv = np.diag(1 / S) if S.ndim == 1 else np.diag([1/s if s > epsilon else 0 for s in S])
    
    # Expandir `S_inv` para que tenga las dimensiones correctas
    S_inv_expanded = np.zeros_like(E_in.T)
    np.fill_diagonal(S_inv_expanded, [1/s if s > epsilon else 0 for s in S])
    
    return Vh.T @ S_inv_expanded @ U.T

M_PR = E_out @ robust_pinv(E_in)
M_PR

array([[ 0.25      +0.10355339j, -0.10355339+0.25j      ],
       [ 0.60355339+0.25j      , -0.25      +0.60355339j]])

In [157]:
# Método de Tikhonov Regularizado (Ridge Regression) 
from scipy.linalg import solve

alpha = 1e-3  # Parámetro de regularización
M_TR = E_out @ np.linalg.inv(E_in.T @ E_in + alpha * np.eye(E_in.shape[1])) @ E_in.T
M_TR 

array([[ 250.        +103.55339059j, -103.55339059+250.j        ],
       [ 603.55339059+250.j        , -250.        +603.55339059j]])

In [159]:
#  Descomposición QR (Robusta para matrices mal condicionadas)
Q, R = np.linalg.qr(E_in)
M_QR = E_out @ np.linalg.inv(R) @ Q.T
M_QR

array([[ 0.25      +0.10355339j, -0.10355339+0.25j      ],
       [ 0.60355339+0.25j      , -0.25      +0.60355339j]])

In [160]:
# Método de Levenberg-Marquardt (Preciso y estable en sistemas no lineales o inestables)
from scipy.optimize import least_squares

# Función de error desglosada en partes reales e imaginarias
def residuals(M_flat):
    M = M_flat.reshape(2, 2)  # Reconstruir la matriz
    error = M @ E_in - E_out
    return np.concatenate([error.real.ravel(), error.imag.ravel()])  # Aplanar el error

# Suposición inicial
initial_guess = np.random.rand(4)  # 4 elementos para una matriz 2x2

# Método de Levenberg-Marquardt
result = least_squares(residuals, initial_guess, method='lm')

# Reconstruir la matriz resultante
M_LM = result.x.reshape(2, 2)
M_LM

array([[0.5       , 0.20710678],
       [1.20710678, 0.5       ]])

In [161]:
# Reducción del Condicionamiento mediante Escalado (Para evitar pérdida de precisión)
scale_factor = np.linalg.norm(E_in)
M_RE = (E_out / scale_factor) @ np.linalg.pinv(E_in / scale_factor)
M_RE

array([[0.25      +0.10355339j, 0.10355339-0.25j      ],
       [0.60355339+0.25j      , 0.25      -0.60355339j]])